# w261 Final Project - Clickthrough Rate Prediction

25   
Justin Trobec, Jeff Li, Sonya Chen, Karthik Srinivasan
Spring 2021, section 5, Team 25

## Table of Contents

* __Section 1__ - Question Formulation
* __Section 2__ - Algorithm Explanation
* __Section 3__ - EDA & Challenges
* __Section 4__ - Algorithm Implementation
* __Section 5__ - Course Concepts

In [4]:
## imports

from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType, ShortType, DateType, BooleanType, BinaryType
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

In [5]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project/"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/airlines/,airlines/,0
dbfs:/mnt/mids-w261/datasets_final_project/airlines_data/,airlines_data/,0
dbfs:/mnt/mids-w261/datasets_final_project/allstate-claims-severity.zip,allstate-claims-severity.zip,51204863
dbfs:/mnt/mids-w261/datasets_final_project/dac.tar.gz,dac.tar.gz,4576820670
dbfs:/mnt/mids-w261/datasets_final_project/kdd-cup-2014-predicting-excitement-at-donors-choose.zip,kdd-cup-2014-predicting-excitement-at-donors-choose.zip,971133938
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/,parquet_airlines_data/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data_3m/,parquet_airlines_data_3m/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data_6m/,parquet_airlines_data_6m/,0
dbfs:/mnt/mids-w261/datasets_final_project/porto-seguro-safe-driver-prediction.zip,porto-seguro-safe-driver-prediction.zip,80247571
dbfs:/mnt/mids-w261/datasets_final_project/walmart-recruiting-trip-type-classification.zip,walmart-recruiting-trip-type-classification.zip,11510035


In [6]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/"))

path,name,size
dbfs:/mnt/mids-w261/10/,10/,0
dbfs:/mnt/mids-w261/BLE_Project/,BLE_Project/,0
dbfs:/mnt/mids-w261/DEMO10/,DEMO10/,0
dbfs:/mnt/mids-w261/DEMO11/,DEMO11/,0
dbfs:/mnt/mids-w261/DEMO8/,DEMO8/,0
dbfs:/mnt/mids-w261/DEMO9/,DEMO9/,0
dbfs:/mnt/mids-w261/HW2/,HW2/,0
dbfs:/mnt/mids-w261/HW3/,HW3/,0
dbfs:/mnt/mids-w261/HW4/,HW4/,0
dbfs:/mnt/mids-w261/HW5/,HW5/,0


In [7]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project/weather_data"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/teamjvml1/,teamjvml1/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather-miss.parquet/,weather-miss.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2015a.parquet/,weather2015a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2016a.parquet/,weather2016a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2017a.parquet/,weather2017a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2018a.parquet/,weather2018a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2019a.parquet/,weather2019a.parquet/,0


In [8]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2015.parquet/,2015.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2016.parquet/,2016.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2017.parquet/,2017.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2018.parquet/,2018.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2019.parquet/,2019.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/airlines_size_test.parquet/,airlines_size_test.parquet/,0


In [9]:
airlines = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/201*.parquet")
display(airlines.sample(False, 0.00001))

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,DIV_AIRPORT_LANDINGS,DIV_REACHED_DEST,DIV_ACTUAL_ELAPSED_TIME,DIV_ARR_DELAY,DIV_DISTANCE,DIV1_AIRPORT,DIV1_AIRPORT_ID,DIV1_AIRPORT_SEQ_ID,DIV1_WHEELS_ON,DIV1_TOTAL_GTIME,DIV1_LONGEST_GTIME,DIV1_WHEELS_OFF,DIV1_TAIL_NUM,DIV2_AIRPORT,DIV2_AIRPORT_ID,DIV2_AIRPORT_SEQ_ID,DIV2_WHEELS_ON,DIV2_TOTAL_GTIME,DIV2_LONGEST_GTIME,DIV2_WHEELS_OFF,DIV2_TAIL_NUM,DIV3_AIRPORT,DIV3_AIRPORT_ID,DIV3_AIRPORT_SEQ_ID,DIV3_WHEELS_ON,DIV3_TOTAL_GTIME,DIV3_LONGEST_GTIME,DIV3_WHEELS_OFF,DIV3_TAIL_NUM,DIV4_AIRPORT,DIV4_AIRPORT_ID,DIV4_AIRPORT_SEQ_ID,DIV4_WHEELS_ON,DIV4_TOTAL_GTIME,DIV4_LONGEST_GTIME,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
2019,2,6,21,5,2019-06-21,DL,19790,DL,N994AT,1853,13485,1348502,33485,MSN,"Madison, WI",WI,55,Wisconsin,45,10397,1039707,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,600,551,-9.0,0.0,0.0,-1,0600-0659,12.0,603,840,7.0,913,847,-26.0,0.0,0.0,-2,0900-0959,0.0,null,0.0,133.0,116.0,97.0,1.0,707.0,3,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,21,5,2019-06-21,AS,19930,AS,N588AS,1787,12892,1289208,32575,LAX,"Los Angeles, CA",CA,6,California,91,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,1055,1050,-5.0,0.0,0.0,-1,1000-1059,26.0,1116,1343,7.0,1345,1350,5.0,5.0,0.0,0,1300-1359,0.0,null,0.0,170.0,180.0,147.0,1.0,954.0,4,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,13,4,2019-06-13,DL,19790,DL,N904DN,2344,10693,1069302,30693,BNA,"Nashville, TN",TN,47,Tennessee,54,10397,1039707,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,1240,1359,79.0,79.0,1.0,5,1200-1259,24.0,1423,1602,6.0,1501,1608,67.0,67.0,1.0,4,1500-1559,0.0,null,0.0,81.0,69.0,39.0,1.0,214.0,1,0.0,0.0,0.0,0.0,67.0,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,21,5,2019-06-21,UA,19977,UA,N33209,732,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,10849,1084905,30849,BZN,"Bozeman, MT",MT,30,Montana,84,940,934,-6.0,0.0,0.0,-1,0900-0959,31.0,1005,1119,5.0,1128,1124,-4.0,0.0,0.0,-1,1100-1159,0.0,null,0.0,108.0,110.0,74.0,1.0,524.0,3,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,5,3,2019-06-05,B6,20409,B6,N639JB,927,11618,1161802,31703,EWR,"Newark, NJ",NJ,34,New Jersey,21,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,823,807,-16.0,0.0,0.0,-2,0800-0859

In [10]:
airlines.printSchema()

root
-- YEAR: integer (nullable = true)
-- QUARTER: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY_OF_MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: string (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
-- OP_CARRIER: string (nullable = true)
-- TAIL_NUM: string (nullable = true)
-- OP_CARRIER_FL_NUM: integer (nullable = true)
-- ORIGIN_AIRPORT_ID: integer (nullable = true)
-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
-- ORIGIN_CITY_MARKET_ID: integer (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- ORIGIN_STATE_FIPS: integer (nullable = true)
-- ORIGIN_STATE_NM: string (nullable = true)
-- ORIGIN_WAC: integer (nullable = true)
-- DEST_AIRPORT_ID: integer (nullable = true)
-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
-- DEST_CITY_MARKET_ID: integer (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- DEST_STATE_FIPS: integer (nullable = true)
-- DEST_STATE_NM: string (nullable = true)
-- DEST_WAC: integer (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_TIME: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: integer (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- ARR_TIME: integer (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: integer (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CANCELLED: double (nullable = true)
-- CANCELLATION_CODE: string (nullable = true)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: integer (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)
-- FIRST_DEP_TIME: integer (nullable = true)
-- TOTAL_ADD_GTIME: double (nullable = true)
-- LONGEST_ADD_GTIME: double (nullable = true)
-- DIV_AIRPORT_LANDINGS: integer (nullable = true)
-- DIV_REACHED_DEST: double (nullable = true)
-- DIV_ACTUAL_ELAPSED_TIME: double (nullable = true)
-- DIV_ARR_DELAY: double (nullable = true)
-- DIV_DISTANCE: double (nullable = true)
-- DIV1_AIRPORT: string (nullable = true)
-- DIV1_AIRPORT_ID: integer (nullable = true)
-- DIV1_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV1_WHEELS_ON: integer (nullable = true)
-- DIV1_TOTAL_GTIME: double (nullable = true)
-- DIV1_LONGEST_GTIME: double (nullable = true)
-- DIV1_WHEELS_OFF: integer (nullable = true)
-- DIV1_TAIL_NUM: string (nullable = true)
-- DIV2_AIRPORT: string (nullable = true)
-- DIV2_AIRPORT_ID: integer (nullable = true)
-- DIV2_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV2_WHEELS_ON: integer (nullable = true)
-- DIV2_TOTAL_GTIME: double (nullable = true)
-- DIV2_LONGEST_GTIME: double (nullable = true)
-- DIV2_WHEELS_OFF: integer (nullable = true)
-- DIV2_TAIL_NUM: string (nullable = true)
-- DIV3_AIRPORT: string (nullable = true)
-- DIV3_AIRPORT_ID: integer (nullable = true)
-- DIV3_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV3_WHEELS_ON: integer (nullable = true)
-- DIV3_TOTAL_GTIME: double (nullable = true)
-- DIV3_LONGEST_GTIME: double (nullable = true)
-- DIV3_WHEELS_

In [11]:
f'{airlines.count():,}'

Out[7]: '31,746,841'

In [12]:
#### For the first phase of the project you will focus on flights departing from two major US airports (ORD (Chicago O'Hare) and ATL (Atlanta) for the first quarter of 2015 (that is about 160k flights)

## 161507 rows
airline_dataset = airlines.filter("(ORIGIN == 'ORD' or ORIGIN == 'ATL') AND YEAR == 2015 AND QUARTER == 1").distinct().take(10)


#display(airline_dataset.sample(False, 0.00001))

In [13]:
airline_dataset

Out[9]: [Row(YEAR=2015, QUARTER=1, MONTH=2, DAY_OF_MONTH=13, DAY_OF_WEEK=5, FL_DATE='2015-02-13', OP_UNIQUE_CARRIER='AA', OP_CARRIER_AIRLINE_ID=19805, OP_CARRIER='AA', TAIL_NUM='N3DLAA', OP_CARRIER_FL_NUM=395, ORIGIN_AIRPORT_ID=13930, ORIGIN_AIRPORT_SEQ_ID=1393003, ORIGIN_CITY_MARKET_ID=30977, ORIGIN='ORD', ORIGIN_CITY_NAME='Chicago, IL', ORIGIN_STATE_ABR='IL', ORIGIN_STATE_FIPS=17, ORIGIN_STATE_NM='Illinois', ORIGIN_WAC=41, DEST_AIRPORT_ID=14843, DEST_AIRPORT_SEQ_ID=1484304, DEST_CITY_MARKET_ID=34819, DEST='SJU', DEST_CITY_NAME='San Juan, PR', DEST_STATE_ABR='PR', DEST_STATE_FIPS=72, DEST_STATE_NM='Puerto Rico', DEST_WAC=3, CRS_DEP_TIME=830, DEP_TIME=823, DEP_DELAY=-7.0, DEP_DELAY_NEW=0.0, DEP_DEL15=0.0, DEP_DELAY_GROUP=-1, DEP_TIME_BLK='0800-0859', TAXI_OUT=16.0, WHEELS_OFF=839, WHEELS_ON=1436, TAXI_IN=8.0, CRS_ARR_TIME=1502, ARR_TIME=1444, ARR_DELAY=-18.0, ARR_DELAY_NEW=0.0, ARR_DEL15=0.0, ARR_DELAY_GROUP=-2, ARR_TIME_BLK='1500-1559', CANCELLED=0.0, CANCELLATION_CODE=None, DIVERTED=0.0, CRS_ELAPSED_TIME=272.0, ACTUAL_ELAPSED_TIME=261.0, AIR_TIME=237.0, FLIGHTS=1.0, DISTANCE=2072.0, DISTANCE_GROUP=9, CARRIER_DELAY=None, WEATHER_DELAY=None, NAS_DELAY=None, SECURITY_DELAY=None, LATE_AIRCRAFT_DELAY=None, FIRST_DEP_TIME=None, TOTAL_ADD_GTIME=None, LONGEST_ADD_GTIME=None, DIV_AIRPORT_LANDINGS=0, DIV_REACHED_DEST=None, DIV_ACTUAL_ELAPSED_TIME=None, DIV_ARR_DELAY=None, DIV_DISTANCE=None, DIV1_AIRPORT=None, DIV1_AIRPORT_ID=None, DIV1_AIRPORT_SEQ_ID=None, DIV1_WHEELS_ON=None, DIV1_TOTAL_GTIME=None, DIV1_LONGEST_GTIME=None, DIV1_WHEELS_OFF=None, DIV1_TAIL_NUM=None, DIV2_AIRPORT=None, DIV2_AIRPORT_ID=None, DIV2_AIRPORT_SEQ_ID=None, DIV2_WHEELS_ON=None, DIV2_TOTAL_GTIME=None, DIV2_LONGEST_GTIME=None, DIV2_WHEELS_OFF=None, DIV2_TAIL_NUM=None, DIV3_AIRPORT=None, DIV3_AIRPORT_ID=None, DIV3_AIRPORT_SEQ_ID=None, DIV3_WHEELS_ON=None, DIV3_TOTAL_GTIME=None, DIV3_LONGEST_GTIME=None, DIV3_WHEELS_OFF=None, DIV3_TAIL_NUM=None, DIV4_AIRPORT=None, DIV4_AIRPORT_ID=None, DIV4_AIRPORT_SEQ_ID=None, DIV4_WHEELS_ON=None, DIV4_TOTAL_GTIME=None, DIV4_LONGEST_GTIME=None, DIV4_WHEELS_OFF=None, DIV4_TAIL_NUM=None, DIV5_AIRPORT=None, DIV5_AIRPORT_ID=None, DIV5_AIRPORT_SEQ_ID=None, DIV5_WHEELS_ON=None, DIV5_TOTAL_GTIME=None, DIV5_LONGEST_GTIME=None, DIV5_WHEELS_OFF=None, DIV5_TAIL_NUM=None),
 Row(YEAR=2015, QUARTER=1, MONTH=2, DAY_OF_MONTH=2, DAY_OF_WEEK=1, FL_DATE='2015-02-02', OP_UNIQUE_CARRIER='AA', OP_CARRIER_AIRLINE_ID=19805, OP_CARRIER='AA', TAIL_NUM='N3GHAA', OP_CARRIER_FL_NUM=354, ORIGIN_AIRPORT_ID=13930, ORIGIN_AIRPORT_SEQ_ID=1393003, ORIGIN_CITY_MARKET_ID=30977, ORIGIN='ORD', ORIGIN_CITY_NAME='Chicago, IL', ORIGIN_STATE_ABR='IL', ORIGIN_STATE_FIPS=17, ORIGIN_STATE_NM='Illinois', ORIGIN_WAC=41, DEST_AIRPORT_ID=12953, DEST_AIRPORT_SEQ_ID=1295302, DEST_CITY_MARKET_ID=31703, DEST='LGA', DEST_CITY_NAME='New York, NY', DEST_STATE_ABR='NY', DEST_STATE_FIPS=36, DEST_STATE_NM='New York', DEST_WAC=22, CRS_DEP_TIME=1635, DEP_TIME=1657, DEP_DELAY=22.0, DEP_DELAY_NEW=22.0, DEP_DEL15=1.0, DEP_DELAY_GROUP=1, DEP_TIME_BLK='1600-1659', TAXI_OUT=19.0, WHEELS_OFF=1716, WHEELS_ON=1950, TAXI_IN=59.0, CRS_ARR_TIME=1938, ARR_TIME=2049, ARR_DELAY=71.0, ARR_DELAY_NEW=71.0, ARR_DEL15=1.0, ARR_DELAY_GROUP=4, ARR_TIME_BLK='1900-1959', CANCELLED=0.0, CANCELLATION_CODE=None, DIVERTED=0.0, CRS_ELAPSED_TIME=123.0, ACTUAL_ELAPSED_TIME=172.0, AIR_TIME=94.0, FLIGHTS=1.0, DISTANCE=733.0, DISTANCE_GROUP=3, CARRIER_DELAY=5.0, WEATHER_DELAY=0.0, NAS_DELAY=49.0, SECURITY_DELAY=0.0, LATE_AIRCRAFT_DELAY=17.0, FIRST_DEP_TIME=None, TOTAL_ADD_GTIME=None, LONGEST_ADD_GTIME=None, DIV_AIRPORT_LANDINGS=0, DIV_REACHED_DEST=None, DIV_ACTUAL_ELAPSED_TIME=None, DIV_ARR_DELAY=None, DIV_DISTANCE=None, DIV1_AIRPORT=None, DIV1_AIRPORT_ID=None, DIV1_AIRPORT_SEQ_ID=None, DIV1_WHEELS_ON=None, DIV1_TOTAL_GTIME=None, DIV1_LONGEST_GTIME=None, DIV1_WHEELS_OFF=None, DIV1_TAIL_NUM=None, DIV2_AIRPORT=None, DIV2_AIRPORT_ID=None, DIV2_AIRPORT_SEQ_ID=None, DIV2_WHEELS_ON=None, DIV2_TOTAL_GTIME=None, DIV2_LONGEST_GTIME=No

In [14]:
display(airlines.describe())

summary,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,DIV_AIRPORT_LANDINGS,DIV_REACHED_DEST,DIV_ACTUAL_ELAPSED_TIME,DIV_ARR_DELAY,DIV_DISTANCE,DIV1_AIRPORT,DIV1_AIRPORT_ID,DIV1_AIRPORT_SEQ_ID,DIV1_WHEELS_ON,DIV1_TOTAL_GTIME,DIV1_LONGEST_GTIME,DIV1_WHEELS_OFF,DIV1_TAIL_NUM,DIV2_AIRPORT,DIV2_AIRPORT_ID,DIV2_AIRPORT_SEQ_ID,DIV2_WHEELS_ON,DIV2_TOTAL_GTIME,DIV2_LONGEST_GTIME,DIV2_WHEELS_OFF,DIV2_TAIL_NUM,DIV3_AIRPORT,DIV3_AIRPORT_ID,DIV3_AIRPORT_SEQ_ID,DIV3_WHEELS_ON,DIV3_TOTAL_GTIME,DIV3_LONGEST_GTIME,DIV3_WHEELS_OFF,DIV3_TAIL_NUM,DIV4_AIRPORT,DIV4_AIRPORT_ID,DIV4_AIRPORT_SEQ_ID,DIV4_WHEELS_ON,DIV4_TOTAL_GTIME,DIV4_LONGEST_GTIME,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
count,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31674354,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31274521,31269545,31269545,31269545,31269545,31746841,31260424,31260429,31244917,31244917,31746841,31244919,31176201,31176201,31176201,31176201,31746841,31746841,489947,31746841,31746677,31178799,31178799,31746841,31746841,31746841,5799114,5799114,5799114,5799114,5799114,203262,203260,203260,31746838,78108,66118,66119,78096,81641,81641,81641,81640,81641,81641,66537,66539,687,691,691,687,687,687,277,277,7,7,7,7,7,7,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
mean,2017.1512498204152,2.5174877084620797,6.552106365480585,15.749554640727876,3.9346285509162944,null,null,19949.20515190787,null,8806.54128440367,2339.5700956514065,12668.724409461716,1266875.803290192,31729.315288031336,null,null,null,26.35374732245013,null,54.91906164774001,12668.666651116562,1266870.0274082704,31729.2951808339,null,null,null,26.354102948384693,null,54.919218135750896,1330.0884999550035,1334.2122192375064,9.855285614165476,12.909587811399238,0.18207943223989986,0.036036885090588944,null,16.830789563186986,1356.9563268309594,1464.4766360877195,7.5604571777227,1488.9034405659447,1468.8957719173475,4.615475952313754,12.966188215170925,0.18601095752494026,-0.20968071125792395,null,0.015432937091284137,null,0.0024603392822611864,143.2167191860742,138.22906985609035,113.8502422431345,1.0,823.2170183483768,3.7652922065537164,19.98459350859459,3.2259498606166392,15.44036813209742,0.08916793151505557,25.364284785572416,1330.4193110369868,37.360331595001476,36.78451244711207,0.003480000118436992,0.8465074000102423,381.64725188299707,234.13669293243998,42.599236836713786,null,12717.885204737815,1271791.6214034616,1509.759015188633,35.31460908122145,28.555541945836037,1540.3529164224417,8805.0,null,12468.658465991317,1246869.2836468886,1384.3318777292577,22.676855895196507,18.86754002911208,1367.2454873646209,null,null,13244.285714285714,1324431.4285714286,1475.2857142857142,15.857142857142858,13.428571428571429,853.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
stddev,1.4316532810215459,1.1053295681781918,3.3994302561415655,8.774238088354572,1.991763538747173,null,null,383.19196153192814,null

### Based on examining the data above, we can determine the following regarding the data below:

#### Categorical

- OP_UNIQUE_CARRIER
- OP_CARRIER_AIRLINE_ID
- OP_CARRIER
- TAIL_NUM

#### Continuous variables


#### Numerical fields

#### Null, or fields with missing data

# Weather
https://data.nodc.noaa.gov/cgi-bin/iso?id=gov.noaa.ncdc:C00532

In [17]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project/weather_data"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/teamjvml1/,teamjvml1/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather-miss.parquet/,weather-miss.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2015a.parquet/,weather2015a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2016a.parquet/,weather2016a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2017a.parquet/,weather2017a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2018a.parquet/,weather2018a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2019a.parquet/,weather2019a.parquet/,0


In [18]:
weather = spark.read.option("header", "true")\
                    .parquet(f"dbfs:/mnt/mids-w261/datasets_final_project/weather_data/*.parquet")

f'{weather.count():,}'

Out[4]: '630,904,436'

In [19]:
display(weather.where('DATE =="DATE"'))

STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AW1,GA1,GA2,GA3,GA4,GE1,GF1,KA1,KA2,MA1,MD1,MW1,MW2,OC1,OD1,OD2,REM,EQD,AW2,AX4,GD1,AW5,GN1,AJ1,AW3,MK1,KA4,GG3,AN1,RH1,AU5,HL1,OB1,AT8,AW7,AZ1,CH1,RH3,GK1,IB1,AX1,CT1,AK1,CN2,OE1,MW5,AO1,KA3,AA3,CR1,CF2,KB2,GM1,AT5,AY2,MW6,MG1,AH6,AU2,GD2,AW4,MF1,AA1,AH2,AH3,OE3,AT6,AL2,AL3,AX5,IB2,AI3,CV3,WA1,GH1,KF1,CU2,CT3,SA1,AU1,KD2,AI5,GO1,GD3,CG3,AI1,AL1,AW6,MW4,AX6,CV1,ME1,KC2,CN1,UA1,GD5,UG2,AT3,AT4,GJ1,MV1,GA5,CT2,CG2,ED1,AE1,CO1,KE1,KB1,AI4,MW3,KG2,AA2,AX2,AY1,RH2,OE2,CU3,MH1,AM1,AU4,GA6,KG1,AU3,AT7,KD1,GL1,IA1,GG2,OD3,UG1,CB1,AI6,CI1,CV2,AZ2,AD1,AH1,WD1,AA4,KC1,IA2,CF3,AI2,AT1,GD4,AX3,AH4,KB3,CU1,CN4,AT2,CG1,CF1,GG1,MV2,CW1,GG4,AB1,AH5,CN3


In [20]:
TEMP = weather[['STATION','DATE','SOURCE','LATITUDE','LONGITUDE','ELEVATION','NAME','REPORT_TYPE','CALL_SIGN','QUALITY_CONTROL']]

In [21]:
display(TEMP.describe())

summary,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL
count,626994336,548964297,630904436,630904436,630904436,630904436,630904436,630904436,630904436
mean,5.900842663765688E10,4.945690792346738,36.82263507723234,-36.70028299582569,369.43297812257043,null,null,99999.0,null
stddev,3.2730239773023808E10,1.3644544181414673,22.617392665563013,78.7181043400128,534.40947703988,null,null,0.0,null
min,10000199999,1,-90.0,-179.999,-999.9,,CRN05,,V020
max,A5125600451,8,83.65,179.998,7070.0,"ZYRYANKA, RS",SY-MT,TXKF,V030


In [22]:
weather.printSchema()

root
-- STATION: string (nullable = true)
-- DATE: timestamp (nullable = true)
-- SOURCE: short (nullable = true)
-- LATITUDE: double (nullable = true)
-- LONGITUDE: double (nullable = true)
-- ELEVATION: double (nullable = true)
-- NAME: string (nullable = true)
-- REPORT_TYPE: string (nullable = true)
-- CALL_SIGN: string (nullable = true)
-- QUALITY_CONTROL: string (nullable = true)
-- WND: string (nullable = true)
-- CIG: string (nullable = true)
-- VIS: string (nullable = true)
-- TMP: string (nullable = true)
-- DEW: string (nullable = true)
-- SLP: string (nullable = true)
-- AW1: string (nullable = true)
-- GA1: string (nullable = true)
-- GA2: string (nullable = true)
-- GA3: string (nullable = true)
-- GA4: string (nullable = true)
-- GE1: string (nullable = true)
-- GF1: string (nullable = true)
-- KA1: string (nullable = true)
-- KA2: string (nullable = true)
-- MA1: string (nullable = true)
-- MD1: string (nullable = true)
-- MW1: string (nullable = true)
-- MW2: string (nullable = true)
-- OC1: string (nullable = true)
-- OD1: string (nullable = true)
-- OD2: string (nullable = true)
-- REM: string (nullable = true)
-- EQD: string (nullable = true)
-- AW2: string (nullable = true)
-- AX4: string (nullable = true)
-- GD1: string (nullable = true)
-- AW5: string (nullable = true)
-- GN1: string (nullable = true)
-- AJ1: string (nullable = true)
-- AW3: string (nullable = true)
-- MK1: string (nullable = true)
-- KA4: string (nullable = true)
-- GG3: string (nullable = true)
-- AN1: string (nullable = true)
-- RH1: string (nullable = true)
-- AU5: string (nullable = true)
-- HL1: string (nullable = true)
-- OB1: string (nullable = true)
-- AT8: string (nullable = true)
-- AW7: string (nullable = true)
-- AZ1: string (nullable = true)
-- CH1: string (nullable = true)
-- RH3: string (nullable = true)
-- GK1: string (nullable = true)
-- IB1: string (nullable = true)
-- AX1: string (nullable = true)
-- CT1: string (nullable = true)
-- AK1: string (nullable = true)
-- CN2: string (nullable = true)
-- OE1: string (nullable = true)
-- MW5: string (nullable = true)
-- AO1: string (nullable = true)
-- KA3: string (nullable = true)
-- AA3: string (nullable = true)
-- CR1: string (nullable = true)
-- CF2: string (nullable = true)
-- KB2: string (nullable = true)
-- GM1: string (nullable = true)
-- AT5: string (nullable = true)
-- AY2: string (nullable = true)
-- MW6: string (nullable = true)
-- MG1: string (nullable = true)
-- AH6: string (nullable = true)
-- AU2: string (nullable = true)
-- GD2: string (nullable = true)
-- AW4: string (nullable = true)
-- MF1: string (nullable = true)
-- AA1: string (nullable = true)
-- AH2: string (nullable = true)
-- AH3: string (nullable = true)
-- OE3: string (nullable = true)
-- AT6: string (nullable = true)
-- AL2: string (nullable = true)
-- AL3: string (nullable = true)
-- AX5: string (nullable = true)
-- IB2: string (nullable = true)
-- AI3: string (nullable = true)
-- CV3: string (nullable = true)
-- WA1: string (nullable = true)
-- GH1: string (nullable = true)
-- KF1: string (nullable = true)
-- CU2: string (nullable = true)
-- CT3: string (nullable = true)
-- SA1: string (nullable = true)
-- AU1: string (nullable = true)
-- KD2: string (nullable = true)
-- AI5: string (nullable = true)
-- GO1: string (nullable = true)
-- GD3: string (nullable = true)
-- CG3: string (nullable = true)
-- AI1: string (nullable = true)
-- AL1: string (nullable = true)
-- AW6: string (nullable = true)
-- MW4: string (nullable = true)
-- AX6: string (nullable = true)
-- CV1: string (nullable = true)
-- ME1: string (nullable = true)
-- KC2: string (nullable = true)
-- CN1: string (nullable = true)
-- UA1: string (nullable = true)
-- GD5: string (nullable = true)
-- UG2: string (nullable = true)
-- AT3: string (nullable = true)
-- AT4: string (nullable = true)
-- GJ1: string (nullable = true)
-- MV1: string (nullable = true)
-- GA5: string (nullable = true)
-- CT2: string (nullable = true)
-- CG2: string

In [23]:
display(weather.sample(False, 0.00001))

STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AW1,GA1,GA2,GA3,GA4,GE1,GF1,KA1,KA2,MA1,MD1,MW1,MW2,OC1,OD1,OD2,REM,EQD,AW2,AX4,GD1,AW5,GN1,AJ1,AW3,MK1,KA4,GG3,AN1,RH1,AU5,HL1,OB1,AT8,AW7,AZ1,CH1,RH3,GK1,IB1,AX1,CT1,AK1,CN2,OE1,MW5,AO1,KA3,AA3,CR1,CF2,KB2,GM1,AT5,AY2,MW6,MG1,AH6,AU2,GD2,AW4,MF1,AA1,AH2,AH3,OE3,AT6,AL2,AL3,AX5,IB2,AI3,CV3,WA1,GH1,KF1,CU2,CT3,SA1,AU1,KD2,AI5,GO1,GD3,CG3,AI1,AL1,AW6,MW4,AX6,CV1,ME1,KC2,CN1,UA1,GD5,UG2,AT3,AT4,GJ1,MV1,GA5,CT2,CG2,ED1,AE1,CO1,KE1,KB1,AI4,MW3,KG2,AA2,AX2,AY1,RH2,OE2,CU3,MH1,AM1,AU4,GA6,KG1,AU3,AT7,KD1,GL1,IA1,GG2,OD3,UG1,CB1,AI6,CI1,CV2,AZ2,AD1,AH1,WD1,AA4,KC1,IA2,CF3,AI2,AT1,GD4,AX3,AH4,KB3,CU1,CN4,AT2,CG1,CF1,GG1,MV2,CW1,GG4,AB1,AH5,CN3
70305500109,2016-03-21T14:56:00.000+0000,4,58.983,-159.05,15.8,"MANOKOTAK AIRPORT, AK US",FM-15,99999,V020,"040,1,N,0067,1","22000,5,9,N","016093,1,9,9","-0017,1","-0044,1","10003,1",,,,,,"9,AGL ,+99999,+99999","04,99,5,99,9,99,9,02134,1,99,9,99,9",,,"09997,1,99999,9","9,9,003,1,+999,9",,,,,,MET104METAR PAMB 211456Z AUTO 04013KT 10SM SCT070 M02/M04 A2952 RMK AO2 SLP003 60000 T10171044 58003 FZRANO $=,Q01 00008PRCP03,,,"2,99,1,+02134,1,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"03,0000,2,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
70305800110,2016-10-09T12:56:00.000+0000,7,56.017,-161.167,4.3,"NELSON LAGOON AIRPORT, AK US",FM-15,PAOU,V020,"150,5,N,0093,5","22000,5,9,N","016093,5,N,5","+0094,5","+0050,5","10041,5",,"00,5,+99999,9,99,9",,,,,"00,99,1,99,9,99,9,99999,9,99,9,99,9",,,"10041,5,10036,5",,,,,,,MET09010/09/16 03:56:02 METAR PAOU 091256Z 15018KT 10SM CLR 09/05 A2965 RMK AO2 SLP041 T00940050,,,,"0,99,1,+99999,9,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
70308025713,2016-03-07T09:03:00.000+0000,4,57.15528,-170.22222,10.7,"ST PAUL ISLAND AIRPORT, AK US",FM-16,99999,V020,"020,1,N,0057,1","00732,1,9,N","004800,1,9,9","-0011,1","-0033,1","99999,9",,,,,,"9,AGL ,+99999,+99999","99,99,9,99,9,99,9,00274,1,99,9,99,9",,,"09905,1,99999,9",,"10,1","71,1",,,,MET109SPECI PASN 070903Z 02011KT 4800 -SN BR FEW009 SCT016 OVC024 M01/M03 A2925 RMK P0000 T10111033 $ VIA AUTODIAL=,,,,"1,99,1,+00274,1,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"2,99,1,+00488,1,9",,,"01,0000,2,1",,,,,,,,,,,,,,,,,,,,,"4,99,1,+00732,1,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
99999994082,2016-06-28T06:20:00.000+0000,null,40.2446,-108.9677,1847.7,"DINOSAUR 2 E, CO US",CRN05,99999,V020,"999,9,9,9999,9","99999,9,9,N","999999,9,9,9","+0241,1","+9999,9","99999,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"05,+0241,1,0,0129,1,0",,,,,"+0241,1,0",,,,,"05,0000,9,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"+0241,1,0",,,,,,,"+04360,1,0",,,,,,,,,,,,,,,,,,"+0241,1,0","+04394,1,0",,,"99,-07",,,,,,,,,,,,,,,,,,,,,,,,,"05,+00000,1,0",,,,,,,,,,,,,,,,,,,,,"+04476,1,0",,,,"11500,1,0,30100,1,0",,,,
99999994085,2016-01-29T20:25:00.000+0000,null,44.0194,-100.353,647.4,"PIERRE 24 S, SD US",CRN05,99999,V020,"999,9,9,9999,9","99999,9,9,N","999999,9,9,9","+0039,1","+9999,9","99999,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"05,+0038,1,0,0774,1,0",,,,,"+0038,1,0",,,,,"05,0000,9,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"+0039,1,0",,,,,,,"+03029,1,0",,,,,,,,,,,,,,,,,,"+0039,1,0","+03052,1,0",,,"99,-06",,,,,,,,,,,,,,,,,,,,,,,,,"05,+00000,1,0",,,,,,,,,,,,,,,,,,,,,"+03095,1,0",,,,"12240,1,0,29370,1,0",,,,
99999994085,2016-06-19T06:05:00.000+0000,null,44.0194,-100.353,647.4,"PIERRE 24 S, SD US",CRN05,99999,V020,"999,9,9,9999,9","99999,9,9,N","999999,9,9,9","+0221,1","+9999,9","99999,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"05,+0221,1,0,0781,1,0",,,,,"+0221,1,0",,,,,"05,0000,9,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"+0221,1,0",,,,,,,"+05309,1,0",,,,,,,,,,,,,,,,,,"+0221,1,0","+05380,1,0",,,"99,-06",,,,,,,,,,,,,,,,,,,,,,,,,"05,+00000,1,0",,,,,,,,,,,,,,,,,,,,,"+05428,1,0",,,,"11510,1,0,29830,1,0",,,,


# Stations

In [25]:
stations = spark.read.option("header", "true").csv("dbfs:/mnt/mids-w261/DEMO8/gsod/stations.csv.gz")

In [26]:
display(stations)

usaf,wban,name,country,state,call,lat,lon,elev,begin,end
007018,99999,WXPOD 7018,null,null,null,0,0,+7018.0,20110309,20130730
007026,99999,WXPOD 7026,AF,null,null,0,0,+7026.0,20120713,20170822
007070,99999,WXPOD 7070,AF,null,null,0,0,+7070.0,20140923,20150926
008268,99999,WXPOD8278,AF,null,null,32.95,65.567,+1156.7,20100519,20120323
008307,99999,WXPOD 8318,AF,null,null,0,0,+8318.0,20100421,20100421
010016,99999,RORVIK/RYUM,NO,null,null,64.85,11.233,+0014.0,19870116,19910806
010017,99999,FRIGG,NO,null,ENFR,59.98,2.25,+0048.0,19880320,20050228
010071,99999,LONGYEARBYEN,SV,null,null,78.217,15.583,+0037.0,20050210,20050210
010190,99999,SVARTTANGEN,NO,null,null,77.517,20.817,+0020.0,20100825,20140523
010303,99999,TROMSO/SKATTURA,NO,null,null,69.7,19.017,+0014.0,20140522,20150108


In [27]:
from pyspark.sql import functions as f
#stations.where(f.col('name').contains('JAN MAYEN NOR NAVY'))
display(stations.where(f.col('name').contains('CHICAGO')))

usaf,wban,name,country,state,call,lat,lon,elev,begin,end
719573,99999,LITTLE CHICAGO,CA,null,null,67.2,-130.217,+0063.0,19910911,19930615
725344,99999,CHICAGO / CALUMET COAST GUARD,US,null,null,41.72,-87.53,+0180.0,19940303,19960630
725346,94866,CHICAGO/MEIGS,US,IL,KCGX,41.867,-87.6,+0181.0,19730101,20080618
725346,99999,CHICAGO/MEIGS,US,IL,KCGX,41.867,-87.6,+0181.0,20000101,20030409
999999,94866,CHICAGO MEIGS FIELD,US,IL,CGX,41.867,-87.617,+0180.1,19660101,19700701
724958,99999,PORT CHICAGO,US,CA,null,38.03,-122.03,+0012.0,19750916,19960607
999999,14819,CHICAGO MIDWAY AP,US,IL,KMDW,41.786,-87.752,+0188.4,19480101,19721231
999999,94846,CHICAGO OHARE INTL AP,US,IL,KORD,41.995,-87.934,+0205.4,19700101,19721231
725337,99999,GARY CHICAGO,US,IN,KGYY,41.617,-87.4,+0180.0,19810126,20051231
725347,99999,CHICAGO/WAUKEGAN,US,IL,KUGN,42.417,-87.867,+0222.0,19890421,20051231


Some notes on joining:
----------------------

- Stations dataset has `usaf` and `wban` columns, which when treated as strings and concatenated map to the weather dataset's `STATION` column.
- Stations dataset has the `call` field, which may be the best bet for joining weather<->station<->airlines(depart/arrival). A challenge with `call` is that the stations table generally has an additional letter, which might relate to the country (I see K used for use airports, C used for Canadian, etc.) However, it also looks like the airlines table only has entries for US domestic flights, in which case we can just always prepend a `K` for the call.
- note that for ORD/ATL, `stations` has two different entries...these have different `usaf` fields but identical `wban`. Also, the elevation seems to be different.
- We'll need to join on time between flights and weather reports as well.

In [29]:
display(stations.where(f.col('call').contains('KORD')))

usaf,wban,name,country,state,call,lat,lon,elev,begin,end
999999,94846,CHICAGO OHARE INTL AP,US,IL,KORD,41.995,-87.934,+0205.4,19700101,19721231
725300,94846,CHICAGO O'HARE INTERNATIONAL,US,IL,KORD,41.995,-87.934,+0201.8,19461001,20190304


In [30]:
display(stations.where(f.col('call').contains('KATL')))

usaf,wban,name,country,state,call,lat,lon,elev,begin,end
999999,13874,ATLANTA HARTSFIELD INTL AP,US,GA,KATL,33.63,-84.442,+0312.7,19450101,19721231
722190,13874,HARTSFIELD-JACKSON ATLANTA IN,US,GA,KATL,33.63,-84.442,+0307.9,19730101,20190304


In [31]:
stations.select('name').distinct().count()

Out[19]: 26497

In [32]:
display(stations.select('name').distinct())

name
HATTFJELLDAL-KRUTA
COLLAFIRTH HILL
WINDY HEAD
ST ATHAN
ARESKUTAN
RYDAL
FLASH-IN-STAFFORD
EPINAL-DOGNEVILLE
FET I EIDFJORD
SAINT LEOCADIE


In [33]:
weather.select('NAME').distinct().count()

Out[21]: 14857

In [34]:
display(weather.select('name').distinct())

name
"PANDAMATENGA, BC"
"VESLJANA, RS"
"NAVRONGO, GH"
"WATKINSVILLE 5 SSE, GA US"
"LANGRES, FR"
"TORREY 7 E, UT US"
"HOHENFELS ARMY AIR FIELD, GM"
"DANMARKSHAVN PORT, GL"
"VALKENBURG, NL"
"ADDO ELEPHANT PARK, SF"


# __Section 1__ - Question Formulation

# __Section 2__ - Algorithm Explanation

# __Section 3__ - EDA & Challenges

# __Section 4__ - Algorithm Implementation

# __Section 5__ - Course Concepts